# Concepts fondateurs
Exercices du cours

# 11.1 Abstraction (1 UC)
## 11.1.2 Classe abstraite / m´ethode abstraite

In [6]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @property
    def alimentation(self):
        return self._aliment

    @alimentation.setter
    def alimentation(self, aliment):
        if aliment in self.diet:
            self._aliment = aliment
        else:
            raise ValueError(f"Cet animal ne mange de {aliment}.")

    @property
    @abstractmethod
    def diet(self):
        pass

    @abstractmethod
    def nourrir(self, time):
        pass

class Lion(Animal):
    @property
    def diet(self):
        return ["cheval", "gazelle", "buffle"]

    def nourrir(self, time):
        print(f"Le lion mange de la viande de {self._aliment} à {time}")

class Serpent(Animal):
    @property
    def diet(self):
        return ["grenouille", "lapin"]

    def nourrir(self, time):
        print(f"Le serpent mange de la viande de {self._aliment} à {time}")
    
leo = Lion()
leo.alimentation = "buffle"
leo.nourrir("12h00")
jo = Serpent()
jo.alimentation = "grenouille"
jo.nourrir("12h20")

leo.alimentation = "carottes"
leo.nourrir("12h00")


Le lion mange de la viande de buffle à 12h00
Le serpent mange de la viande de grenouille à 12h20


ValueError: Cet animal ne mange de carottes.

In [7]:
import random

class TableauTrieAbstrait():

    def __init__(self):
        self.liste = []

    def plusGrand(self,a,b):
        raise NotImplementedError()

    def inserer(self, element):
        i = 0 
        while i < len(self.liste) and self.plusGrand(element, self.liste[i]):
            i += 1        
        self.liste.insert(i, element)

class TableauTrieEntiers(TableauTrieAbstrait):
    
    def __init__(self):
        super().__init__()

    def plusGrand(self,a,b):
        return a > b

class TableauTrieChaines(TableauTrieAbstrait):
    
    def __init__(self):
        super().__init__()

    def plusGrand(self,a,b):
        return a > b

In [8]:
entiers = TableauTrieEntiers()

for j in range(5):
    entiers.inserer(random.randint(1, 50))

print(entiers.liste)

cars = TableauTrieChaines()

for j in range(5):
    car = chr(random.randint(ord('a'), ord('z')))
    cars.inserer(car)

print(cars.liste)

[14, 15, 25, 32, 46]
['b', 'b', 'b', 'i', 't']


## Exercice suppl´ementaire 11.1. Vie parisienne

## Exercice supplémentaire 11.2

Un simulateur fait ”vivre” trois personnages ´economiques que sont :
1. Le Paysan
2. Le Meunier
3. Le Boulanger

Les classes à construire sont :    

| Paysan                   |  Le Meunier                               | Le Boulanger                                                                             |
|:-------------------------|:------------------------------------------|:-----------------------------------------------------------------------------------------|
|Mange du pain pour vivre  |Mange du pain pour vivre                   |Mange du pain (de son propre stock) pour vivre                                            |
|                          |Reçoit du blé                              |Reçoit de la farine                                                                       |
|Produit du blé            |Produit de la farine en consommant du blé  |Fabrique du pain en consommant de la farine                                               |
|Livre du blé au un menier |Livre de la farine au boulanger            |Livre du pain aux meuniers et aux paysans (en fait, `a tous ceux qui peuvent en recevoir) |
|Reçoit du pain            |Reçoit du pain                             |                                                                                          |




In [51]:
class Personne:

    def __init__(self, metier_name, name):
        self.metier_name = metier_name
        self.nom = name
        self.nb_pains = 0
        self.intermediaires = []

    def ajouter_client(self, intermediaire):
        self.intermediaires.append(intermediaire)
    
    def manger(self):
        # if self.nb_pains > 0:
        #     self.nb_pains -= 1
        # else:
        #     self.nb_pains = -1
        self.nb_pains -= 1
        return self.nb_pains

    def produire(self, nb_produits):
        raise NotImplementedError()
    
    def receptionner(self, livreur, quantite):
        raise NotImplementedError()

    def get_nb_production(self):
        raise NotImplementedError()
    
    def reduce_production(self, nb_produits):
        raise NotImplementedError()

    def livrer(self, nb):
        nb_production = self.get_nb_production()
        if nb_production >= nb:
            nb_par_pers = nb / len(self.intermediaires)
            for i in range(len(self.intermediaires)):
                self.reduce_production(nb_par_pers)
                self.intermediaires[i].receptionner(self.metier_name, nb_par_pers)
            print(self.intitule()+" livrer:", nb)

    def to_string(self):
        return self.intitule()+" nb_pains:" + str(self.nb_pains)

    def intitule(self):
        return self.nom +" "*(7-len(self.nom)) +"("+ self.metier_name+ ") " +" "*(7-len(self.metier_name))

class Paysan(Personne):

    metier_name = "Paysan"
    coefficient = 1

    def __init__(self, name):
        super().__init__(Paysan.metier_name, name)
        self.stock_ble = 0

    def produire(self, nb_produits):
        self.stock_ble += nb_produits
        print(self.intitule()+"produit:",nb_produits, "soit:", self.stock_ble)
        # Livraison par quatre dose, reste va dans le stock
        a_livrer = self.stock_ble // 4
        self.livrer(a_livrer * 4)
        
    def receptionner(self, livreur, quantite):
        if livreur == Boulanger.metier_name:
            self.nb_pains += quantite    

    def get_nb_production(self):
        return self.stock_ble

    def reduce_production(self, nb_produits):
        self.stock_ble -= nb_produits

    def to_string(self):
        return super().to_string() + ", Stock ble ("+str(self.stock_ble)+")"


class Boulanger(Personne):

    metier_name = "Boulanger"
    coefficient = 4

    def __init__(self, name):
        super().__init__(Boulanger.metier_name, name)
        self.stock_farine = 0
        
    def receptionner(self, livreur, quantite):
        if livreur == Meunier.metier_name:
            self.stock_farine += quantite

    def produire(self, nb_produits):
        for i in range(nb_produits):
            # il faut 1/4 de dose de farine pour produire 1 pain
            if self.stock_farine >= 0.25:
                self.nb_pains += 1
                self.stock_farine -= 0.25
            # il n'y a plus assez de farine pour continuer la production
            else:
                break
        print(self.intitule()+"produit:",i, "/ ",nb_produits,"soit:", self.nb_pains)
        print("produit:",self.to_string())
        # on garde une partie pour nourir le boulanger
        nb_a_livrer = self.nb_pains // (len(self.intermediaires) + 1)
        # il est toutefois possible de livrer ce qui a été produit
        self.livrer(nb_a_livrer * len(self.intermediaires))
        
    def get_nb_production(self):
        return self.nb_pains

    def reduce_production(self, nb_produits):
        self.nb_pains -= nb_produits
    
    def to_string(self):
        return super().to_string() + ", Stock farine ("+str(self.stock_farine)+")"

class Meunier(Personne):

    metier_name = "Meunier"
    coefficient = 0.25

    def __init__(self, name):
        super().__init__(Meunier.metier_name, name)
        self.stock_ble = 0
        self.stock_farine = 0
        
    def receptionner(self, livreur, quantite):
        if livreur == Paysan.metier_name:
            self.stock_ble += quantite
        elif livreur == Boulanger.metier_name:
            self.nb_pains += quantite

    def reduce_production(self, nb_produits):
        self.stock_farine -= nb_produits

    def get_nb_production(self):
        return self.stock_farine

    def produire(self, nb_produits):
        # il faut 4 doses de blé pour produire 1 dose de farine
        for i in range(nb_produits):
            if self.stock_ble >= 4:
                self.stock_ble -= 4
                self.stock_farine += 1
            # il n'y a plus assez de blé pour continuer la production
            else:
                break
        print(self.intitule()+"produit:",i, "/ ",nb_produits,"soit:", self.stock_farine)
        # il est toutefois possible de livrer ce qui a été produit
        self.livrer(self.stock_farine)

    def to_string(self):
        return super().to_string() + ", Stock farine ("+str(self.stock_farine)+"), Stock ble ("+str(self.stock_ble)+")"


In [56]:
jean = Paysan("Jean")
michel = Meunier("Michel")
louis = Boulanger("Louis")

jean.ajouter_client(michel)
michel.ajouter_client(louis)
louis.ajouter_client(jean)
louis.ajouter_client(michel)

commercants = [jean, michel, louis]

nb_tours = random.randint(1, 10)

for i in range(nb_tours):
    for commercant in commercants:
        # on nourrit les commerçant dans la seconde boucle pour qu'il y ait un 1er circuit complet
        if i > 0 and commercant.manger() < 0:
            print(commercant.intitule(), "est mort de faim !!!")
            # Suppression de la liste des commerçant
            commercants.remove(commercant)
            break
        if commercant.metier_name == Boulanger.metier_name:
            commercant.produire(random.randint(4, 40))
        else:
            commercant.produire(random.randint(1, 10))   
        # Affichage du résultat
        print(commercant.to_string())
    print(i, "/",nb_tours, "-----------------------------------------------------", i, "/",nb_tours)

# Pour que le Garbage collector puisse passer quand les instances ne seront plus nécessaires
jean = None
michel = None
louis = None

Jean   (Paysan)  produit: 6 soit: 6
Jean   (Paysan)   livrer: 4
Jean   (Paysan)   nb_pains:0, Stock ble (2.0)
Michel (Meunier) produit: 1 /  4 soit: 1
Michel (Meunier)  livrer: 1
Michel (Meunier)  nb_pains:0, Stock farine (0.0), Stock ble (0.0)
Louis  (Boulanger) produit: 4 /  18 soit: 4
produit: Louis  (Boulanger)  nb_pains:4, Stock farine (0.0)
Louis  (Boulanger)  livrer: 2
Louis  (Boulanger)  nb_pains:2.0, Stock farine (0.0)
0 / 10 ----------------------------------------------------- 0 / 10
Jean   (Paysan)  produit: 9 soit: 11.0
Jean   (Paysan)   livrer: 8.0
Jean   (Paysan)   nb_pains:0.0, Stock ble (3.0)
Michel (Meunier) produit: 2 /  9 soit: 2.0
Michel (Meunier)  livrer: 2.0
Michel (Meunier)  nb_pains:0.0, Stock farine (0.0), Stock ble (0.0)
Louis  (Boulanger) produit: 8 /  29 soit: 9.0
produit: Louis  (Boulanger)  nb_pains:9.0, Stock farine (0.0)
Louis  (Boulanger)  livrer: 6.0
Louis  (Boulanger)  nb_pains:3.0, Stock farine (0.0)
1 / 10 ------------------------------------------